# prologue

### set up notebook and load package

In [1]:
# load what we need
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# data management

### datasets

Any dataset can be turned into a container by invoking the constructor found in the module structures.py, the only mandatory parameters are an object that can be input to pandas.DataFrame and a class column name. Your object needs either its own column names, or these should be passed as a list to the var_names parameter.

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm. If these pre-packaged sets have more than 10,000 rows then we've included some downsampled versions. You can see what there is by doing:

In [2]:
# ignore the builtins, np, pd, urllib and cfg. The rest are dataset constructors that will load specific datasets.
if False:
    dir(ds)

You can see more information about each dataset with something like this for the adult dataset:

In [3]:
if False:
    print(ds.adult().spiel)

An example of one dataset is given below. Note that the random_state propagates through other functions via the meta_data and is easily updated to allow alternative runs:

In [4]:
if False:
    cardio = ds.cardio(random_state=123, project_dir=project_dir)
    meta_data = cardio.get_meta()
    meta_data['random_state']

The list of datasets used in "CHIRPS: Explaining Random Forest Classification" is:

In [5]:
if False:
    datasets = [
            ds.adult_small_samp,
            ds.bankmark_samp,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery_samp,
            ds.rcdv_samp,
           ]

The list of datasets used in "Explaining Multi-class AdaBoost Classification in Medical Applications" is:

In [6]:
if False:
    datasets = [
                ds.cardio,
                ds.breast,
                ds.mhtech14,
                ds.readmit_samp,
                ds.thyroid
               ]

### Standardising train-test splitting across environments
To compare with methods in different environments, as we did in "CHIRPS: Explaining Random Forest Classification," you can maintain the same dataset splits. The train test data is split with the one-time random_state_splits and the splits are saved to csv in the project folders. This way you can do same model with different data splits, or same data splits with different model. This is what's happening behind the scenes:

In [7]:
##### do not run this here
# train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
# tt = mydata.tt_split(train_index, test_index)

All you need here is the following, to take the same train_index, test_index numbers to an external .csv file

In [8]:
# writes external files
if False:
    rp.export_data_splits(datasets=datasets, project_dir=project_dir, random_state_splits=random_state_splits)

# Experimental Runs
From here on, everything is automated behind the scenes to perform runs across several datasets and comparing different algorithms with CHIRPS. If you want to run CHIRPS and control the parameters and view results and performance directly, look at the CHIRPS Examples Notebook.

In [9]:
# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
chirps_explanation_async=True

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 10000
start_instance = 11 # here can opt to start at a specific instance, diagnostic if something crashes

# CHOOSE ONE
# model = 'RandomForest'
model = 'AdaBoost1' # SAMME
# model = 'AdaBoost2' # SAMME.R
# model = 'GBM'

# CHOOSE ONE OR MORE
do_CHIRPS = False
do_Anchors = False
do_dfrgTrs = False
do_lore = True

# list the dataset constructors you want to include
datasets = [
#             ds.adult_small_samp,
#             ds.bankmark_samp,
#             ds.car,
            ds.cardio,
#             ds.credit,
#             ds.german,
#             ds.lending_tiny_samp,
#             ds.nursery_samp,
#             ds.rcdv_samp,
            ds.breast,
            # ds.cervicalh,
            ds.mhtech14,
            ds.readmit_samp,
            ds.thyroid
           ]
# datasets = [ds.cervicalh] # here can opt for just one but it must be a list, e.g. datasets = [datasets[0]]

# location to save results
# project_dir = '/datadisk/whiteboxing/examples'
project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'

# set the random_state for various tasks. This is not the same as random.seed(), not system wide.
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

# How much messaging to print to the screen?
verbose = True

# CHIRPS default parameters - see papers for details
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.1 # pruning deterioration tolerance paramater

# this is here if you want to pass parameters to the methods
def benchmark_wrapper(do_CHIRPS=True, do_Anchors=False, do_dfrgTrs=False, do_lore=False, **control):

    if do_CHIRPS:
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                    'Kmax' : 10, 'restart' : 100, 'maxitr' : 20})
        rp.do_benchmarking(benchmark_items, verbose, **control)

    if do_lore:
        control.update({'method' : 'lore'})
        rp.do_benchmarking(benchmark_items, verbose, **control)

# this is here to pass parameters to the model training and further parameters to CHIRPS
tuning = {'grid' : None, 'override' : False}
if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           do_raw=True, do_discretise=do_Anchors,
                                           start_instance=start_instance, verbose=verbose)

    kwargs = {'support_paths' : 0.1, 'alpha_paths' : 0.5, 'disc_path_bins' : 4,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=True, do_discretise=do_Anchors,
                                       start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.1, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=True, do_discretise=do_Anchors,
                                       start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.01, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 8, 'disc_path_eqcounts' : True,
                 'score_func' : 1, 'weighting' : 'kldiv',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    

    
else: # GBM - not fully implemented yet
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)

Preprocessing cardio data and model for cardio with random state = 123
Split data into main train-test and build forest
NSP
Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.9503
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 800, 'random_state': 123}


Preprocessing breast data and model for breast with random state = 123
Split data into main train-test and build forest
mb
Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.9799
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criteri

C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.8286
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 200, 'random_state': 123}


Preprocessing readmit_samp data and model for readmit_samp with random state = 123
Split data into main train-test and build forest
readmitted
Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.6200
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            mi

C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.9671
Best parameters:{'algorithm': 'SAMME', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 1200, 'random_state': 123}


lore benchmark
Running lore on each instance and collecting results
Working on lore for instance 1189
starting generate genetic neighbourhood
Sat Sep 14 15:26:38 2019
in sso


...

C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


unable to create genetic neighbourhood
starting generate random neighbourhood
endings generate neighbourhood
Sat Sep 14 15:47:04 2019

Working on lore for instance 1374
starting generate genetic neighbourhood
Sat Sep 14 15:47:06 2019


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


in sdo
[[1.21000000e+02 0.00000000e+00 2.93511298e-02 8.69565217e-03
  4.17616612e-03 0.00000000e+00 0.00000000e+00 2.40000000e+01
  5.02470441e-01 0.00000000e+00 4.90000000e+00 4.87054509e+01
  8.50000000e+01 1.49000000e+02 5.00000000e+00 0.00000000e+00
  1.33000000e+02 1.49000000e+02 1.25462952e+02 4.00000000e+00
  0.00000000e+00]
 [1.21000000e+02 0.00000000e+00 2.93511298e-02 8.69565217e-03
  4.17616612e-03 0.00000000e+00 0.00000000e+00 2.40000000e+01
  5.02470441e-01 0.00000000e+00 4.90000000e+00 4.87054509e+01
  8.50000000e+01 1.49000000e+02 5.00000000e+00 0.00000000e+00
  1.33000000e+02 1.49000000e+02 1.25462952e+02 4.00000000e+00
  0.00000000e+00]]


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
stop

## Sensitivity Analysis

In [ ]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

# prepare data
datasets = [
            ds.adult_small_samp,
            ds.bankmark_samp,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery_samp,
            ds.rcdv_samp
           ]
# datasets = [datasets[0]] # for testing can just choose one


# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1 # prune rule terms if loss of precision no greater than delta

forest_walk_async=True
chirps_explanation_async=True

n_instances = 10000

# model = 'RandomForest'
# model = 'AdaBoost1'
model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = False
do_dfrgTrs = False

# here can opt for just one, e.g. [datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
# project_dir = '/datadisk/whiteboxing'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}

# for troubleshooting, can isolate an instance and do like
# bi_copy['cardio']['main']['ds_container'].current_row_test = 110

if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance, verbose=verbose)
    
    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.1, 0.05], 18), 2)
    weighting = np.repeat(['chisq', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'disc_path_eqcounts' : False,
                        'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : 'majority',
                        'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : 'rf_sensitivity'}

        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
        support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
        n_kwargs = 36
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        support_paths = np.tile(np.tile([0.005, 0.002, 0.001], 16), 2)
        n_kwargs = 72
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 6), 2)
    disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 6), 2)
    weighting = np.tile(np.repeat(['chisq', 'kldiv', 'nothing'], 12), 2)
    which_trees = np.repeat(['majority', 'conf_weighted'], 36)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : wchtr,
                        'delta' : delta} 
    for k, dpb, dpeq, w, sp, wchtr \
                   in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

    for kwargs in kwargs_grid: # range(24, 36): #
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : save_sensitivity_path}
 
        rp.do_benchmarking(bi_copy, verbose, **control)